In [53]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split

In [54]:
#Helper functions for data processing
def card_to_num(card):
    raw_rank = card[:-1]
    
    ranks = {
        '2' : 0,
        '3' : 1,
        '4' : 2, 
        '5' : 3,
        '6' : 4, 
        '7' : 5, 
        '8' : 6, 
        '9' : 7, 
        '10': 8, 
        'J' : 9, 
        'Q' : 10, 
        'K' : 11, 
        'A': 12
    }

    return ranks[raw_rank]

def hand_to_list(hand):
    '''Takes hand like KH-AC and outputs list of card numbers'''
    hand_list_1 = hand.split("-")
    hand_list_2 = [card_to_num(card) for card in hand_list_1]
    return hand_list_2

result_mapping = {
    'hit' : 0,
    'stand' : 1,
    'double down' : 2
}

batch_size = 32

# Defining Dataset Class
class Blackjack_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [55]:
split_or_not_raw_df = pd.read_csv('CSVs/split_or_not.csv')

In [56]:
# Cleaned split_or_not
split_or_not_raw_df['dealer_upcard'] = split_or_not_raw_df['dealer_upcard'].apply(card_to_num)
split_or_not_raw_df['player_hand'] = split_or_not_raw_df['player_hand'].apply(hand_to_list)
split_or_not_raw_df['player_hand'] = split_or_not_raw_df['player_hand'].apply(lambda hand: hand[0])
split_or_not_df = split_or_not_raw_df.rename(columns = {'player_hand':'player_upcard'})

# Turning into tensor matrices
# split_or_not
x1 = torch.tensor(split_or_not_df['player_upcard'].values, dtype=torch.float32).unsqueeze(1)
x2 = torch.tensor(split_or_not_df['dealer_upcard'].values, dtype=torch.float32).unsqueeze(1)
y = torch.tensor(split_or_not_df['result'].values, dtype=torch.float32).view(-1,1)

X = torch.cat([x1,x2], dim=1)

split_or_not_dataset = Blackjack_Dataset(X,y)
train_sn, test_sn = train_test_split(split_or_not_dataset, test_size=0.2) #I might have to do this earlier, on the dataframe, but this was easier so let's see if it works

sn_train_dataloader = DataLoader(train_sn, batch_size=batch_size, shuffle=True)
sn_test_dataloader = DataLoader(test_sn, batch_size=batch_size, shuffle=True)

In [67]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()

    for batch, (X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Printing Training Update on every 100th batch
        if (batch + 1) % 100 == 0: 
            loss = loss.item()
            current = batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    #Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True

    with torch.no_grad():
        for X, y in dataloader: 
            pred = model(X)
            probs = torch.sigmoid(pred)
            preds = (probs >= 0.5).float()
            correct += (preds.view(-1) == y.view(-1)).float().sum()
            test_loss += loss_fn(pred, y).item()


    test_loss /= num_batches
    correct /= size 
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [70]:
class sn_NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

sn_model = sn_NeuralNetwork()

learning_rate = 0.0005 
epochs = 20

loss_fn = nn.BCEWithLogitsLoss()
sn_optimizer = torch.optim.SGD(sn_model.parameters(), lr=learning_rate)

In [71]:
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------")
    train_loop(sn_train_dataloader, sn_model, loss_fn, sn_optimizer)
    test_loop(sn_test_dataloader, sn_model, loss_fn)
print("Done!")

Epoch 1
---------------------------
loss: 0.520788  [ 3200/141921]
loss: 0.641984  [ 6400/141921]
loss: 0.502605  [ 9600/141921]
loss: 0.555203  [12800/141921]
loss: 0.564010  [16000/141921]
loss: 0.583664  [19200/141921]
loss: 0.635257  [22400/141921]
loss: 0.638516  [25600/141921]
loss: 0.618868  [28800/141921]
loss: 0.580581  [32000/141921]
loss: 0.607461  [35200/141921]
loss: 0.556880  [38400/141921]
loss: 0.536216  [41600/141921]
loss: 0.559943  [44800/141921]
loss: 0.610080  [48000/141921]
loss: 0.592350  [51200/141921]
loss: 0.534066  [54400/141921]
loss: 0.566548  [57600/141921]
loss: 0.406015  [60800/141921]
loss: 0.470939  [64000/141921]
loss: 0.461016  [67200/141921]
loss: 0.697420  [70400/141921]
loss: 0.626229  [73600/141921]
loss: 0.566435  [76800/141921]
loss: 0.642122  [80000/141921]
loss: 0.693759  [83200/141921]
loss: 0.655650  [86400/141921]
loss: 0.645181  [89600/141921]
loss: 0.615251  [92800/141921]
loss: 0.616370  [96000/141921]
loss: 0.554116  [99200/141921]
los